In [1]:
# Crea un netcdf con el viento a 925 hPa alineado con respecto
# a la fecha del onset de la temporada de lluvias.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

# Datos básicos.

path_d = "../data/ERA5/"
path_r = "../results/onset/"

# Cantidad de días a promediar.
d = 20

# Variable, nivel de presión y región.
vars = [ "u", "v", "sst", "olr", "gp", "sp" ]
levels = [ "925", "200" ]
region = [ "mexico" ]

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
shift_2 = pd.read_csv(path_r + "shift_withdrawal_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift
delta_2 = shift_2.max()
shift_2 = shift_2.max() - shift_2

In [2]:
# viento 925 hPa.

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[0] + "_daily.grib"
u = xr.open_dataset( path_d + fname, engine = "cfgrib" )
u = u.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
u = ( u.sel( time = ~( (u.time.dt.month == 2) & (u.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
u = ( u.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
u.to_netcdf( path_r + "onset" + vars[0] + "_" + levels[0] + "_mean_" 
    + str(d) + "_dias.nc" )

# Viento meridional.
fname = region[0] + "_" + vars[1] + "_" + levels[0] + "_daily.grib"
v = xr.open_dataset( path_d + fname, engine = "cfgrib" )
v = v.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
v = ( v.sel( time = ~( (v.time.dt.month == 2) & (v.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
v = ( v.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
v.to_netcdf( path_r + "onset" + vars[1] + "_" + levels[0] + "_mean_" 
    + str(d) + "_dias.nc" )

# Unimos en un solo netcdf.
ds = xr.merge( ( u, v ) )

# Onset.
# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_wind_" + levels[0] + "_mean_" 
    + str(d) + "_dias.nc" )

# Withdrawal.
# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift_2.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "withdrawal_composite_wind_" + levels[0]
    + "_mean_" + str(d) + "_dias.nc" )

composite

RuntimeError: NetCDF: HDF error

In [3]:
# viento 200 hPa.

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[1] + "_daily.grib"
u = xr.open_dataset( path_d + fname, engine = "cfgrib" )
u = u.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
u = ( u.sel( time = ~( (u.time.dt.month == 2) & (u.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
u = ( u.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
u.to_netcdf( path_r + "onset" + vars[0] + "_" + levels[1] + "_mean_" 
    + str(d) + "_dias.nc" )

# Viento meridional.
fname = region[0] + "_" + vars[1] + "_" + levels[1] + "_daily.grib"
v = xr.open_dataset( path_d + fname, engine = "cfgrib" )
v = v.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
v = ( v.sel( time = ~( (v.time.dt.month == 2) & (v.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
v = ( v.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
v.to_netcdf( path_r + "onset" + vars[1] + "_" + levels[1] + "_mean_" 
    + str(d) + "_dias.nc" )

# Unimos en un solo netcdf.
ds = xr.merge( ( u, v ) )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_wind_" + levels[1] + "_mean_" 
    + str(d) + "_dias.nc" )

composite

<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    u            (dayofyear, latitude, longitude) float64 22.01 22.04 ... 0.987
    v            (dayofyear, latitude, longitude) float64 -2.1 -2.046 ... 0.1775

In [4]:
# Geopotencial a 925 hPa.
fname = region[0] + "_" + vars[2] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset" + vars[4] + "_" + levels[0] + "_mean_" 
    + str(d) + "_dias.nc" )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[2] + "_" + levels[0] 
    + "_mean_" + str(d) + "_dias.nc" )


composite

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_sst_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sst          (dayofyear, latitude, longitude) float64 nan nan ... nan nan

In [5]:
# Geopotencial a 200 hPa.
fname = region[0] + "_" + vars[2] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset" + vars[4] + "_" + levels[1] + "_mean_" 
    + str(d) + "_dias.nc" )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[2] + "_" + levels[1] 
    + "_mean_" + str(d) + "_dias.nc" )

composite

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_sst_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sst          (dayofyear, latitude, longitude) float64 nan nan ... nan nan

In [6]:
# Temperatura superficial del océano.
fname = region[0] + "_" + vars[2] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset" + vars[2] + "_mean_"  + str(d) + "_dias.nc" )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[2] + "_mean_" 
    + str(d) + "_dias.nc" )

composite

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_sst_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sst          (dayofyear, latitude, longitude) float64 nan nan ... nan nan

In [2]:
# OLR
fname = region[0] + "_" + vars[3] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset" + vars[3] + "_mean_" + str(d) + "_dias.nc" )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[3] + "_mean_" 
    + str(d) + "_dias.nc" )


composite

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_olr_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    mtnlwrf      (dayofyear, latitude, longitude) float64 -222.2 ... -271.9

In [3]:
# Surface Pressure
fname = region[0] + "_" + vars[5] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[5] + "_mean_" + str(d) + "_dias.nc" )

# Composite de la distribución espacial.
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[2] + "_mean_"
    + str(d) + "_dias.nc" )

composite

<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sp           (dayofyear, latitude, longitude) float64 8.38e+04 ... 1.013e+05